## Object Classification

* Cifar10 데이터셋 사용
* CNN으로 Object Classification

In [ ]:
import torch

from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu' #GPU 사용이 가능하다면 GPU를 사용하고 아니면 CPU를 사용한다.

In [ ]:
# parameters
learning_rate = 0.001 #학습율, 하이퍼파라미터로 학습하는 것이 아니고 입력하는 값
epochs = 10            # 전체 데이터를 1번 학습한다.
batch_size = 100      # 데이터 학습시 데이터의 단위(100개 씩 학습함)
drop_out = 0.25        # 1 epoch 학습할 때 0.25프로 연결을 끊는다. 정규화하는 방법 중의 하나이다.

In [ ]:
# MNIST dataset
transform = transforms.ToTensor()
training_data = datasets.CIFAR10(root='CIFAR_data/', #데이터의 위치
                                 train = True,
                                 transform=transform, #이미지를 tensor에 맞게 조정
                                 download=True) #데이터를 root위치에 다운로드한다. 이미 존재하는 경우 다시 다운하지 않는다.

test_data = datasets.CIFAR10(root='CIFAR_data/',
                             train=False, 
                             transform=transform,
                             download=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)                                    

In [ ]:
class CNN(nn.Module):
    
    def __init__(self):
        super(CNN, self).__init__()
        #첫번째 layer 생성
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size = 5, stride =1, padding = 1), # 입력 채널 개수(컬러이미지), 출력 채널 개수, 건너 띄기 없음, same padding
            nn.ReLU(), #activation function. (비선형함수, 음수는 0으로 바꾼다)
            nn.MaxPool2d(kernel_size = 2, stride = 2), #32x32 -> 16x16x32로 사이즈가 줄어든다
            nn.Dropout(drop_out)) 
        
        #두 번째 layer 생성
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride =1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2), #16x16x64 -> 8x8x64로 사이즈가 줄어든다
            nn.Dropout(drop_out)) 

        #세 번째 layer 생성
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=2, stride =1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2), #8x8x64 -> 4x4x128로 사이즈가 줄어든다
            nn.Dropout(drop_out)) 

        #네 번째 layer 생성
        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=2, stride =1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2), #4x4x128 -> 2x2x256로 사이즈가 줄어든다
            nn.Dropout(drop_out)) 

        #fully connected layer 생성
        self.fc = nn.Linear(2 * 2 * 256, 10, bias = True) # 행렬 데이터를 벡터로 바꿔를 입력하여 10개(0~9)로 출력함
        nn.init.xavier_uniform_(self.fc.weight)
   
   #모델 설계 후 데이터셋이 layer들을 통과할 수 있게 함.
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1) 
        out = self.fc(out)
        return out

In [ ]:
model = CNN().to(device)

In [ ]:
#cost & loss 명시하기 & optimizer
# pytorch는 softmax가 포함이 되어 있기 때문에 생략
criterion = nn.CrossEntropyLoss().to(device)    
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate) #gradient descent할 때 adam 활용

In [ ]:
total_batch = len(train_dataloader)
model.train()
print(total_batch) # total batch값 500출력. (5만개의 데이터가 batch_size인 100으로 나뉜 값)
print('Learning started. It takes sometime.')
for epoch in range(epochs):
    avg_cost = 0

    for X, Y in train_dataloader:
        X = X.to(device) # batch_size(100), 채널(3= 컬러), 32 X 32 사이즈의 이미지 데이터를 입력함
        Y = Y.to(device)

        optimizer.zero_grad()            # optimizer 변수에 포함시킨 매개 변수(weight)들의 기울기(Gradient)를 0으로 초기화
        hypothesis = model(X)            # 가설 설정
        cost = criterion(hypothesis, Y)  # 비용 설정
        cost.backward()                  # 역전파
        optimizer.step()                 # 학습을 통해 계산한 weight, bias, gradient를 최적화함수(optimizer)에에 반영

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

500
Learning started. It takes sometime.
[Epoch:    1] cost = 1.72557235
[Epoch:    2] cost = 1.39580989
[Epoch:    3] cost = 1.24194062
[Epoch:    4] cost = 1.1490159
[Epoch:    5] cost = 1.06548655
[Epoch:    6] cost = 1.00073266
[Epoch:    7] cost = 0.961116552
[Epoch:    8] cost = 0.924154758
[Epoch:    9] cost = 0.897860289
[Epoch:   10] cost = 0.870349586
Learning Finished!


In [ ]:
correct = 0
total = 0
with torch.no_grad(): #gradient 비활성화
    model.eval()
    for X_test, Y_test in test_dataloader:
        outputs = model(X_test.float().to(device))
        _, predicted = torch.max(outputs.data, 1)
        total += Y_test.size(0)
        correct += (predicted == Y_test.to(device)).sum().item()

print(f'Accuracy: {100 * correct // total} %')

Accuracy: 72 %


In [ ]:
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
with torch.no_grad(): #gradient 비활성화
    model.eval()
    for X_test, Y_test in test_dataloader:
        outputs = model(X_test.float().to(device))
        _, predictions = torch.max(outputs.data, 1)
        for label, pred in zip(Y_test.to(device), predictions):
          if label == pred:
              correct_pred[classes[label]] += 1
          total_pred[classes[label]] += 1

for classname, correct_count in correct_pred.items():
    accuracy = 100 * correct_count / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: plane is 69.6 %
Accuracy for class: car   is 83.5 %
Accuracy for class: bird  is 58.6 %
Accuracy for class: cat   is 46.4 %
Accuracy for class: deer  is 67.8 %
Accuracy for class: dog   is 69.4 %
Accuracy for class: frog  is 86.0 %
Accuracy for class: horse is 69.6 %
Accuracy for class: ship  is 90.8 %
Accuracy for class: truck is 78.8 %
